## Determining Optimal Zipcodes to live in - Data Science Capstone Project
### Introduction

Austin, Texas has quickly emerged as one of the fastest growing cities in America. Austin’s population continues to grow, with an anticipated growth rate of 3% year over year. This is not surprising, as people are drawn to the relatively low cost of living, tech-focused industry, outdoorsy lifestyle, and year-round activities that take place in the city. Often dubbed the Silicon Valley of Texas, Austin continues to attract tech giants such as Apple, Google, Indeed and IBM - as well as many startup companies - leading to a robust job market. That combined with the attractive lifestyle that Austin presents has brought thousands of new people to the city and surrounding areas, which has had a massive effect on the real estate market. Though it continues to see positive change throughout the years, the market can be quite intimidating when it comes to deciding the right part of the city to move to, especially for those unfamiliar to the area. Whether one is looking to rent or a buy, it can be both competitive and time-consuming to find the ideal home. 

With this in mind, our goal is to help make this decision easier by determining optimal zip codes for people to consider when finding a new home in the Austin area. In order to achieve this, we will be utilizing Foursquare's API in order to make calls that will give us locational data on restaurants, gyms, companies, and any relevant venues in the surrounding area that could assist or hinder characteristics within the zipcode.

### Data Preparation and Feature Selection

Data for this project was collected from [AustinTexas.gov](http://www.austintexas.gov/page/demographic-data) under the American Community Survey 2017 Profiles for ZIP Codes section. Each zipcode was a HTML download link containing an excel file holding four sheets of data. Each of the four sheets of data were categories containing data related to Demographic, Economic, Social, and Housing characteristics. Data that was labeled as necessary to include was subjective, but important features considered originated primarily from a set of survey responses to features deemed "Important" when considering a place to live. Because of this, not all quantifiable categories were included, and categories projected to explain a small amount of variance in potential machine learning models like race were excluded.

### Data Cleaning

All four sheets were scraped for desired data using a program that iterates through each of the four sheets. In our excel files, our desirable features were under one column, while data related to those features were within adjacent cells. In order to fix this, the program written had to use the Series.isin() function in order to retrieve the names of the features, and their data. This program was reused for all 46 zipcodes, and for the purpose of this project, only 1 call of the program being will be shown, to save the reader time. Once each sheet had been scraped for important features I stored them along with the respective zipcode into a single dataframe to call on later.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import folium
import geocoder

In [2]:
relevant_cols = ['Male', 'Female', 'Median age in years', 'Mean household income', 
                'Family households', 'Nonfamily households', 'Occupied housing units', 
                 'Vacant housing units', 'Average home value', 'Average gross rent']

relevant_cols

['Male',
 'Female',
 'Median age in years',
 'Mean household income',
 'Family households',
 'Nonfamily households',
 'Occupied housing units',
 'Vacant housing units',
 'Average home value',
 'Average gross rent']

In [3]:
def cleanTable(df1, df2, df3, df4, zipcode):
    
    """This program aims to take 4 excel files and a zipcode labeled in the dataframe as inputs, one for each of the sheets within
    an individal zipcode excel file, and organize it into a row within a dataframe."""
    
    #Re-label our column names so the code below is reusable
    df1.columns = ['ACS_Profile_Report', 'Number', 'MOE', 'Pct']
    df2.columns = ['ACS_Profile_Report', 'Number', 'MOE', 'Pct']
    df3.columns = ['ACS_Profile_Report', 'Number', 'MOE', 'Pct']
    df4.columns = ['ACS_Profile_Report', 'Number', 'MOE', 'Pct']
    
    #strip all leading and trailing spaces within the column listed so no data is skipped when iterating
    df1['ACS_Profile_Report'] = df1['ACS_Profile_Report'].str.strip()
    df2['ACS_Profile_Report'] = df2['ACS_Profile_Report'].str.strip()
    df3['ACS_Profile_Report'] = df3['ACS_Profile_Report'].str.strip()
    df4['ACS_Profile_Report'] = df4['ACS_Profile_Report'].str.strip()
    
    #For each dataframe - if the Profile Report column contains the category name we want as an independant column, retrieve it.
    for data in df1['ACS_Profile_Report']:
        if data in relevant_cols:
            df1 = df1.loc[df1['ACS_Profile_Report'].isin(relevant_cols)]
        else:
            continue
    
    for data in df2['ACS_Profile_Report']:
        if data in relevant_cols:
            df2 = df2.loc[df2['ACS_Profile_Report'].isin(relevant_cols)]
        else:
            continue
            
    for data in df3['ACS_Profile_Report']:
        if data in relevant_cols:
            df3 = df3.loc[df3['ACS_Profile_Report'].isin(relevant_cols)]
        else:
            continue
            
    for data in df4['ACS_Profile_Report']:
        if data in relevant_cols:
            df4 = df4.loc[df4['ACS_Profile_Report'].isin(relevant_cols)]
        else:
            continue
            
    new_df = pd.concat([df1,df2,df3,df4])
    new_df['Zipcode'] = zipcode
    new_df.drop_duplicates(inplace=True)
    new_df.drop(columns=['MOE', 'Pct'], inplace=True)
    new_df['Number'] = new_df['Number'].astype('int64')
    new_df = pd.pivot_table(new_df, columns='ACS_Profile_Report', values='Number', index='Zipcode')
    new_df.reset_index()
    
    return new_df

__Importing four excel files, one for each sheet within the excel file, all related to a single zipcode.__

In [4]:
df_two = pd.read_excel('ACS_Profile_78701_v2.xls', sheet_name='Economic')
df_one = pd.read_excel('ACS_Profile_78701_v2.xls', sheet_name='Demographic')
df_three = pd.read_excel('ACS_Profile_78701_v2.xls', sheet_name='Social')
df_four = pd.read_excel('ACS_Profile_78701_v2.xls', sheet_name='Housing')

__Example of what the individual rows look like in their respective columns after being added into the dataframe__

In [5]:
df_78701 = cleanTable(df_one, df_two, df_three, df_four, 78701)
df_78701

ACS_Profile_Report,Average gross rent,Average home value,Family households,Female,Male,Mean household income,Median age in years,Nonfamily households,Occupied housing units,Vacant housing units
Zipcode,,,,,,,,,,
78701,1917,728781,1187,3208,4667,173289,35,3668,4855,1721


__Once all dataframes were concatenated together, dataframe was cleaned up and stored as a csv file under the name Austin_Zipcodes__

In [6]:
df = pd.read_csv('Austin_Zipcodes.csv')
df.head()

,Zipcode,Average gross rent,Average home value,Mean household income,Median age in years,Family households,Nonfamily households,Total Population,Male,Female,Occupied housing units,Vacant housing units
0,78701,1917,728781,173289,35,1187,3668,7875,4667,3208,4855,1721
1,78702,1105,299188,73919,33,4180,4757,22876,11643,11233,8937,902
2,78703,1542,911518,166396,36,4597,5126,20788,10429,10359,9723,1324
3,78704,1309,486017,91223,32,7862,15351,47158,24489,22669,23213,2602
4,78705,1397,482162,41357,21,1201,7169,33075,15932,17143,8370,1966


__Instantiate our Nominatim object to retrieve geographic coordinates for our zipcodes with Austin.__

In [7]:
geolocator = Nominatim(user_agent='austincityscrape')

In [8]:
lat = []
lng = []
for i in df['Zipcode']:
    
    location = geolocator.geocode(i)
    latitude = location.latitude
    lat.append(latitude)
    longitude = location.longitude
    lng.append(longitude)

df['Latitude'] = lat
df['Longitude'] = lng
df.head(10)

,Zipcode,Average gross rent,Average home value,Mean household income,Median age in years,Family households,Nonfamily households,Total Population,Male,Female,Occupied housing units,Vacant housing units,Latitude,Longitude
0,78701,1917,728781,173289,35,1187,3668,7875,4667,3208,4855,1721,30.280466,-97.750088
1,78702,1105,299188,73919,33,4180,4757,22876,11643,11233,8937,902,30.278460,-97.718800
2,78703,1542,911518,166396,36,4597,5126,20788,10429,10359,9723,1324,30.271561,-97.761267
3,78704,1309,486017,91223,32,7862,15351,47158,24489,22669,23213,2602,30.227821,-97.778097
4,78705,1397,482162,41357,21,1201,7169,33075,15932,17143,8370,1966,30.286748,-97.725799
5,78721,1059,215338,52873,34,2458,1739,12447,5885,6562,4197,435,30.272626,-97.684817
6,78722,1355,380270,95443,32,1293,1818,7254,3266,3988,3111,304,30.285703,-97.718354
7,78723,1071,299158,71897,33,6690,5828,33170,16301,16869,12518,1259,30.252530,-97.681176
8,78724,1116,138684,55252,28,5071,1289,24833,12578,12255,6360,394,30.290684,-97.626557
9,78725,1347,139626,66225,35,1709,835,7517,4044,3473,2544,54,30.231742,-97.643690


In [9]:
austin = geolocator.geocode('Austin, TX')
austin_lat = austin.latitude
austin_lng = austin.longitude

print('The coordinates of Austinn are {}, {}.'.format(austin_lat, austin_lng))

The coordinates of Austinn are 30.2711286, -97.7436995.


### Methodology - Exploratory Data Analysis